<div style="padding:25px; background-color:#f8f9fa; border:1px solid #dee2e6; border-radius:10px; line-height:1.6;">
    <h1 style="color:#c5003d; margin-top:0;">🛍️ Case Study: Bon Prix Personalization Lab</h1>
    <hr style="border:0; height:1px; background:#dee2e6;">
    <p style="font-size:1.1em;">
        <b>Die Herausforderung:</b> Kunden im E-Commerce sind von tausenden Rezensionen überfordert. 
        Ein Standard-Rating (Sterne) sagt nichts über die individuellen Bedürfnisse aus.
    </p>
    <ul style="margin-left:20px;">
        <li><b>Ziel:</b> KI-gestützte Zusammenfassungen, die nur das anzeigen, was für die <i>spezifische</i> Kundin wichtig ist.</li>
        <li><b>Technologie:</b> Large Language Models (LLM), NLP, Python & Gradio.</li>
        <li><b>Business-Value:</b> Höhere Conversion-Rate & weniger Retouren durch bessere Passform-Informationen.</li>
    </ul>
</div>

In [1]:
# ==========================================
# 1. WERKZEUGKASTEN & SETUP
# ==========================================
import pandas as pd     # Datenanalyse (Das "Excel" von Python)
import gradio as gr     # Erstellung der Web-Oberfläche
import ollama           # Schnittstelle zum lokalen Sprachmodell (Llama 3)
import os, json, re     # Datei- und Textverarbeitung
import plotly.graph_objects as go  # Interaktive Business-Grafiken
from dotenv import load_dotenv
from langfuse import observe, get_client


# .env Datei laden (Damit wir unsere Keys nicht hardcoden müssen)
load_dotenv()


# Langfuse Credentials
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_BASE_URL = os.getenv("LANGFUSE_BASE_URL")


# Dateipfade (Unsere Datenbasis)
USER_FILE = 'personalization_users_visible.csv' 
REVIEWS_FILE = 'reviews_all_users_in_shop.csv'
CACHE_FILE = "bonprix_project_cache.json"

# Cache-Funktionen (Damit wir nicht für jede Anfrage die KI neu bezahlen/befragen müssen)
def get_cache():
    if not os.path.exists(CACHE_FILE): return {}
    try:
        with open(CACHE_FILE, "r", encoding="utf-8") as f: return json.load(f)
    except: return {}

def set_cache(key, val):
    c = get_cache()
    c[key] = val
    with open(CACHE_FILE, "w", encoding="utf-8") as f: 
        json.dump(c, f, indent=4, ensure_ascii=False)

/Users/niko/Dev/IBM/2025/bonprix/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<div style="padding:15px; background-color:#e3f2fd; border-left:5px solid #2196f3; border-radius:5px;">
    <h3 style="color:#0d47a1; margin-top:0;">📊 Schritt 1: Das "Gedächtnis" füttern</h3>
    <p>Bevor die KI beraten kann, muss sie wissen, mit wem sie spricht. Wir kombinieren zwei Datenquellen:</p>
    <table style="width:100%; border-collapse: collapse; margin-top:10px;">
        <tr style="background-color:rgba(255,255,255,0.5);">
            <td style="padding:8px; border:1px solid #bbdefb;"><b>Kunden-Profile</b></td>
            <td style="padding:8px; border:1px solid #bbdefb;">Historische Rezensionen: Was hat die Kundin früher bemängelt oder gelobt?</td>
        </tr>
        <tr>
            <td style="padding:8px; border:1px solid #bbdefb;"><b>Produkt-Rezensionen</b></td>
            <td style="padding:8px; border:1px solid #bbdefb;">Aktuelles Feedback anderer Kunden zum neuen Wunsch-Produkt.</td>
        </tr>
    </table>
</div>

In [2]:
class BonPrixEngine:
    def __init__(self):
        # 1. Alle Shop-Rezensionen laden
        self.reviews = pd.read_csv(REVIEWS_FILE, sep=';')
        self.reviews.columns = [c.strip() for c in self.reviews.columns]
        
        # 2. User-Profile & Historie aufbauen
        self.users = {}
        if os.path.exists(USER_FILE):
            df_u = pd.read_csv(USER_FILE, sep=';')
            df_u.columns = [c.strip() for c in df_u.columns]
            
            for name in df_u['user_name'].unique():
                if pd.isna(name): continue
                sub = df_u[df_u['user_name'] == name]
                
                # Wir verknüpfen alle alten Rezensionstexte zu einem "Profil-String"
                past_reviews = " | ".join(sub['review_text'].dropna().astype(str).tolist())
                
                # Einfache statistische Merkmale (z.B. Durchschnittsgröße)
                sz_raw = sub['size'].mode()[0] if not sub['size'].mode().empty else "40"
                sz = int(re.search(r'\d+', str(sz_raw)).group()) if re.search(r'\d+', str(sz_raw)) else 38
                
                self.users[name] = {
                    "base_size": sz,
                    "past_reviews": past_reviews # Das "Gedächtnis" für die KI
                }
        
        # 3. Produktkatalog (Top 20 Produkte für die Demo)
        top_p = self.reviews['product_id'].value_counts().head(20).index.tolist()
        self.products = self.reviews[self.reviews['product_id'].isin(top_p)][['product_id', 'short_description']]\
            .drop_duplicates().set_index('product_id')['short_description'].to_dict()

engine = BonPrixEngine()

In [4]:
<div style="padding:20px; background-color:#fff3e0; border:2px dashed #ff9800; border-radius:10px;">
    <h3 style="color:#e65100; margin-top:0;">🧠 Schritt 2: Prompt Engineering (Management via AI)</h3>
    <p>Hier definieren wir das <b>"Briefing"</b> für unser Sprachmodell. Als Manager steuern Sie die KI nicht durch Code, sondern durch präzise Instruktionen:</p>
    <blockquote style="font-style:italic; color:#5d4037; border-left:3px solid #ffb74d; padding-left:15px;">
        "Analysiere, welche Aspekte (Material, Stil, Passform) für diese Kundin laut ihrer Historie wichtig sind und fasse das neue Produkt NUR darauf basierend zusammen."
    </blockquote>
    <p style="font-size:0.9em; margin-top:10px;"><b>Interaktive Aufgabe:</b> Wir werden gleich versuchen, den Tonfall der KI zu ändern. Was passiert, wenn wir die KI anweisen, 'extrem kritisch' zu sein?</p>
</div>

SyntaxError: invalid character '🧠' (U+1F9E0) (2241122232.py, line 2)

In [5]:
def build_student_prompt(user_name, profile, product_name, current_product_reviews):
    """
    Diese Funktion baut das 'Briefing' für die KI.
    Hier fließt das Business-Wissen ein (Worauf soll die KI achten?).
    """
    return f"""
    Die Kundin {user_name} schaut sich gerade das Produkt "{product_name}" an.
    
    HIER SIND DIE REZENSIONEN ZUM AKTUELLEN PRODUKT:
    {current_product_reviews}
    
    DIE KUNDIN SELBST HAT IN DER VERGANGENHEIT FOLGENDE REZENSIONEN GESCHRIEBEN:
    "{profile['past_reviews']}"
    
    AUFGABE:
    Erstelle eine personalisierte Zusammenfassung der Rezensionen zum neuen Produkt für {user_name}.
    Analysiere dafür zuerst, welche Aspekte (z.B. Material, Stil, Nachhaltigkeit, Passform) für {user_name} 
    laut ihren alten Rezensionen besonders wichtig sind. 
    Fasse dann die neuen Rezensionen genau im Hinblick auf diese persönlichen Prioritäten zusammen.
    
    WICHTIG:
    - Wenn die Kundin nie über Größe geschrieben hat, ignoriere das Thema Passform.
    - Konzentriere dich auf ihre Nuancen.
    - Schreibe direkt, ehrlich und in max. 4 Sätzen auf Deutsch.
    """

In [ ]:
# ==========================================
# 3. ANALYSE-LOGIK (Berechnungen & KI-Aufruf)
# ==========================================

langfuse = get_client()


@observe(name="BonPrix_Project", capture_output=False)
def run_analysis(user_name, product_selection):
    if not user_name or not product_selection: 
        return [None]*3 + ["### ⚠️ Bitte links Kundin und Produkt wählen!", ""]
    
    pid = int(product_selection.split(' - ')[0])
    product_name = engine.products.get(pid, "Unbekanntes Produkt")
    profile = engine.users[user_name]
    df = engine.reviews[engine.reviews['product_id'] == pid]
    
    # Wir loggen Metadaten für das Management-Dashboard
    langfuse.update_current_span(
        metadata={
            "customer_name": user_name,
            "product_id": pid,
            "past_reviews_count": len(profile['past_reviews'].split('|'))
        }
    )

    # --- A. VISUALISIERUNG: Passform & Aspekte ---
    txt_all = " ".join(df['review_text'].dropna().astype(str).tolist()).lower()
    
    # Passform-Tacho
    s, l = len(re.findall(r'klein|eng|kurz', txt_all)), len(re.findall(r'groß|weit|lang', txt_all))
    fit_score = (l - s) / (l + s + 1)
    fig_gauge = go.Figure(go.Indicator(mode="gauge+number", value=fit_score, title={'text': "Passform-Tendenz"},
                                       gauge={'axis': {'range': [-1, 1]}, 'bar': {'color': "#c5003d"}}))
    
    # Radar-Chart für Produkt-Eigenschaften
    def get_s(kw): return min(100, int((sum(txt_all.count(w) for w in kw) / (len(df)+1)) * 300))
    aspects = {"Komfort": get_s(["bequem", "weich"]), "Qualität": get_s(["qualität", "stoff"]), 
               "Stil": get_s(["schön", "optik"]), "Preis": get_s(["preis", "günstig"])}
    fig_radar = go.Figure(data=go.Scatterpolar(r=list(aspects.values()), theta=list(aspects.keys()), fill='toself'))
    
    # Sterne-Verteilung
    counts = df['rating'].value_counts().sort_index()
    fig_stars = go.Figure(data=[go.Bar(x=[f"{i}★" for i in counts.index], y=counts.values, marker_color='#f1c40f')])

    for f in [fig_gauge, fig_radar, fig_stars]: 
        f.update_layout(height=280, margin=dict(l=40, r=40, t=40, b=40), template="plotly_white")


    # --- B. KI-PERSONALISIERUNG ---
    reviews_sample = "\n- ".join(df['review_text'].head(15).astype(str).tolist())[:1500]
    final_prompt = build_student_prompt(user_name, profile, product_name, reviews_sample)
    
    cache = get_cache()
    if f"{user_name}_{pid}" in cache:
        return fig_gauge, fig_radar, fig_stars, cache[f"{user_name}_{pid}"], final_prompt
    else:
        try:
            ans = ""
            with langfuse.start_as_current_observation(
                name="ollama-generation",
                as_type="generation",
                model="llama3",
                input=final_prompt
            ) as generation:
                
                generated_text = ollama.chat(model="llama3", messages=[{'role': 'user', 'content': final_prompt}])
                ans = generated_text['message']['content']
                set_cache(f"{user_name}_{pid}", ans)

                generation.update(
                    output=ans,
                    metadata={"engine": "ollama"}
                )

                langfuse.update_current_trace(output=ans)
        except Exception as e:
            langfuse.update_current_span(level="ERROR", status_message=str(e))
            ans = f"### ⚠️ KI-Fehler\nOllama ist nicht gestartet: {e}"

    langfuse.flush()

    return fig_gauge, fig_radar, fig_stars, ans, final_prompt
    
    

<div style="padding:15px; background-color:#e8f5e9; border-left:5px solid #4caf50; border-radius:5px;">
    <h3 style="color:#1b5e20; margin-top:0;">🚀 Schritt 3: Das MVP (Minimum Viable Product)</h3>
    <p>Ein Algorithmus ohne Interface ist für den Fachbereich nutzlos. Mit <b>Gradio</b> bauen wir eine Web-Oberfläche, die zeigt, wie die Lösung im Kundensupport oder in der App aussehen könnte.</p>
    <b>Klicken Sie unten auf den Link, um das Labor zu starten!</b>
</div>

In [7]:
# UI DESIGN
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🛍️ Bon Prix: Personalization Lab")
    
    with gr.Row():
        with gr.Column(scale=1, variant="panel"):
            gr.Markdown("### 🛠️ Konfiguration")
            user_dd = gr.Dropdown(choices=list(engine.users.keys()), value=list(engine.users.keys())[0] if engine.users else None, label="1. Kundin wählen")
            prod_dd = gr.Dropdown(choices=[f"{k} - {v}" for k,v in engine.products.items()], label="2. Produkt wählen")
            btn = gr.Button("🚀 Personalisierte Analyse", variant="primary")
            gr.Markdown("---")
            gr.Markdown("**Info:** Die KI analysiert im Hintergrund die Profile von echten bon prix Kundinnen.")

        with gr.Column(scale=2):
            with gr.Tabs():
                with gr.TabItem("📝 KI-Beratung"):
                    ai_out = gr.Markdown("### 💡 Ergebnis\nWählen Sie links eine Kundin und ein Produkt...")
                    with gr.Accordion("🔍 Prompt-Einsicht (Technik)", open=False):
                        prompt_out = gr.Code(language="markdown")
                
                with gr.TabItem("📊 Statistiken"):
                    with gr.Row():
                        plot_gauge = gr.Plot()
                        plot_radar = gr.Plot()
                    plot_stars = gr.Plot()

    btn.click(run_analysis, inputs=[user_dd, prod_dd], outputs=[plot_gauge, plot_radar, plot_stars, ai_out, prompt_out])

# Start der App
if __name__ == "__main__":
    demo.launch()

/var/folders/ts/0dhd43l91q956_87n7js139h0000gn/T/ipykernel_94747/2165335655.py:2: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
